#Exercise 3. Simple intracellular Ca2+ signaling system


In [1]:
from neuron import h, rxd
import time
from pylab import *
from ipywidgets import widgets, fixed, Layout

In [2]:
%%html
<style type='text/css'>
.widget-inline-hbox .widget-label {
      max-width: 250px;
      min-width: 250px;
}
</style>

In [3]:
#This is the initialization of the RxD system - should be run only once
h.load_file('stdrun.hoc')

dend = h.Section(name='dend')                                 #Create compartment dend
dend.L=1                                                      #Set the dimensions
dend.diam=0.79788                                             #of the compartment
cyt = rxd.Region([dend], name='cyt', nrn_region='i')          #Region (cytosol) where the dynamics will take place

my_volume = 5e-16 #(0.5 um3)                                  #Volume of the compartment (needed for conversion of

Duration = 4000000                                            #Length of the simulation (ms)
tolerance = 1e-5                                              #Tolerance of the adaptive time step integration
Ca_input_onset = 3000000                                      #Onset time of the Ca2+ input
Ca_input_N     = 100                                          #Number of successive inputs per train
Ca_input_freq  = 100                                          #Frequency of successive inputs
Ca_input_dur   = 3.0                                          #Duration of each Ca2+ pulse
Ca_input_flux  = 2000.0                                       #Flux during each pulse (in particles/ms)

initfile = ''
addition = ''
blocked = []
blockeds = []
block_factor = 1.0

#Vector of initial values. Includes more species than what is used in this script, see below for the subset of considered species
initvalues = [7.3e-05, 1.877026, 0.001814, 0.000225, 0.145869, 0.014658, 0.0, 0.0, 2.502689, 0.00049, 1.3e-05, 0.016767, 0.537911, 0.005711, 0.00406, 9e-06, 5.4e-05, 0.00382, 0.002623, 1e-06, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.001508, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000114, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.994911, 5.5e-05, 5e-06, 8.1e-05, 0.003709, 2e-06, 0.0, 0.011482, 0.000297, 0.0, 0.000976, 0.002143, 0.018013, 0.011485, 0.00018, 4e-06, 0.0, 0.00206, 0.000266, 3e-06, 0.021291, 7e-06, 0.001059, 0.000868, 0.0, 0.0, 7e-06, 6e-06, 0.002021, 0.000302, 4.3e-05, 0.0, 0.0, 0.000729, 0.0, 6e-06, 0.001121, 0.000442, 0.0, 0.0, 1.1e-05, 0.00027, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.000314, 0.0, 0.0, 4.5e-05, 0.0, 0.0, 1.0, 0.0, 0.0, 0.002472, 0.0, 0.0, 0.0, 1.019075, 0.001441, 0.0, 0.0, 0.000445, 1.8e-05, 0.0, 0.0, 0.023783, 3.4e-05, 0.0, 0.0, 0.0, 0.0, 0.000293, 0.000402, 0.014994, 0.0, 0.0, 9.4e-05, 0.000327, 0.0, 0.0, 0.001556, 0.000253, 9.2e-05, 0.0, 0.0, 0.000601]

species = ['Ca', 'CaOut', 'CaOutLeak', 'Leak', 'pmca', 'ncx', 'pmcaCa', 'ncxCa', 'CaM', 'CaMCa2', 'CaMCa4', 'CK', 'CKCaMCa4', 'CKpCaMCa4', 'CKp', 'Complex', 'pComplex', 'CKpPP1', 'CKpCaMCa4PP1', 'PP1', 'GluR1', 'GluR1_S831', 'GluR1_CKCam', 'GluR1_CKpCam', 'GluR1_CKp', 'GluR1_S831_PP1']
spec_Ca = rxd.Species(cyt, name='Ca', charge=2, initial=initvalues[0])
spec_CaOut = rxd.Species(cyt, name='CaOut', charge=0, initial=initvalues[1])
spec_CaOutLeak = rxd.Species(cyt, name='CaOutLeak', charge=0, initial=initvalues[2])
spec_Leak = rxd.Species(cyt, name='Leak', charge=0, initial=initvalues[3])
spec_pmca = rxd.Species(cyt, name='pmca', charge=0, initial=initvalues[11])
spec_ncx = rxd.Species(cyt, name='ncx', charge=0, initial=initvalues[12])
spec_pmcaCa = rxd.Species(cyt, name='pmcaCa', charge=0, initial=initvalues[13])
spec_ncxCa = rxd.Species(cyt, name='ncxCa', charge=0, initial=initvalues[14])
spec_CaM = rxd.Species(cyt, name='CaM', charge=0, initial=initvalues[72])
spec_CaMCa2 = rxd.Species(cyt, name='CaMCa2', charge=0, initial=initvalues[73])
spec_CaMCa4 = rxd.Species(cyt, name='CaMCa4', charge=0, initial=initvalues[74])
spec_CK = rxd.Species(cyt, name='CK', charge=0, initial=initvalues[79])
spec_CKCaMCa4 = rxd.Species(cyt, name='CKCaMCa4', charge=0, initial=initvalues[80])
spec_CKpCaMCa4 = rxd.Species(cyt, name='CKpCaMCa4', charge=0, initial=initvalues[81])
spec_CKp = rxd.Species(cyt, name='CKp', charge=0, initial=initvalues[82])
spec_Complex = rxd.Species(cyt, name='Complex', charge=0, initial=initvalues[83])
spec_pComplex = rxd.Species(cyt, name='pComplex', charge=0, initial=initvalues[84])
spec_CKpPP1 = rxd.Species(cyt, name='CKpPP1', charge=0, initial=initvalues[85])
spec_CKpCaMCa4PP1 = rxd.Species(cyt, name='CKpCaMCa4PP1', charge=0, initial=initvalues[86])
spec_PP1 = rxd.Species(cyt, name='PP1', charge=0, initial=initvalues[95])
spec_GluR1 = rxd.Species(cyt, name='GluR1', charge=0, initial=initvalues[100])
spec_GluR1_S831 = rxd.Species(cyt, name='GluR1_S831', charge=0, initial=initvalues[102])
spec_GluR1_CKCam = rxd.Species(cyt, name='GluR1_CKCam', charge=0, initial=initvalues[105])
spec_GluR1_CKpCam = rxd.Species(cyt, name='GluR1_CKpCam', charge=0, initial=initvalues[106])
spec_GluR1_CKp = rxd.Species(cyt, name='GluR1_CKp', charge=0, initial=initvalues[107])
spec_GluR1_S831_PP1 = rxd.Species(cyt, name='GluR1_S831_PP1', charge=0, initial=initvalues[114])

Ca_flux_rate = rxd.Parameter(cyt, initial=0)

#Reactions concerning Ca influx and efflux                                                                                                                                                                                                 
reaction000 = rxd.Reaction(spec_Ca + spec_pmca != spec_pmcaCa, 50.0, 0.007)
reaction001 = rxd.Reaction(spec_pmcaCa > spec_pmca + spec_CaOut, 0.0035)
reaction002 = rxd.Reaction(spec_Ca + spec_ncx != spec_ncxCa, 16.8, 0.0112)
reaction003 = rxd.Reaction(spec_ncxCa > spec_ncx + spec_CaOut, 0.0056)
reaction004 = rxd.Reaction(spec_CaOut + spec_Leak != spec_CaOutLeak, 1.5, 0.0011)
reaction005 = rxd.Reaction(spec_CaOutLeak > spec_Ca + spec_Leak, 0.0011)

#Reactions concerning calmodulin and CaMKII                                                                                                                                                                                                
reaction067 = rxd.Reaction(spec_CaM + spec_Ca != spec_CaMCa2, 6.0, 0.0091)
reaction068 = rxd.Reaction(spec_CaMCa2 + spec_Ca != spec_CaMCa4, 100.0, 1.0)
reaction075 = rxd.Reaction(spec_CaMCa4 + spec_CK != spec_CKCaMCa4, 10.0, 0.003)
reaction076 = rxd.Reaction(spec_CKCaMCa4*2 != spec_Complex, 0.1, 0.01)
reaction077 = rxd.Reaction(spec_CKpCaMCa4 + spec_CKCaMCa4 != spec_pComplex, 0.1, 0.01)
reaction078 = rxd.Reaction(spec_CKpCaMCa4 + spec_Complex > spec_CKpCaMCa4 + spec_pComplex, 0.1)
reaction079 = rxd.Reaction(spec_CKCaMCa4 + spec_Complex > spec_CKCaMCa4 + spec_pComplex, 0.1)
reaction080 = rxd.Reaction(spec_Complex*2 > spec_Complex + spec_pComplex, 10.0)
reaction081 = rxd.Reaction(spec_Complex + spec_pComplex > spec_pComplex*2, 30.0)
reaction082 = rxd.Reaction(spec_CKpCaMCa4 != spec_CaMCa4 + spec_CKp, 8e-07, 10.0)
reaction083 = rxd.Reaction(spec_CKp + spec_PP1 != spec_CKpPP1, 0.004, 0.00034)
reaction084 = rxd.Reaction(spec_CKpPP1 > spec_PP1 + spec_CK, 8.6e-05)
reaction085 = rxd.Reaction(spec_CKpCaMCa4 + spec_PP1 != spec_CKpCaMCa4PP1, 0.004, 0.00034)
reaction086 = rxd.Reaction(spec_CKpCaMCa4PP1 > spec_PP1 + spec_CKCaMCa4, 8.6e-05)

#Reactions concerning AMPA receptors                                                                                                                                                                                                       
reaction100 = rxd.Reaction(spec_GluR1 + spec_CKCaMCa4 != spec_GluR1_CKCam, 0.02224, 0.0016)
reaction101 = rxd.Reaction(spec_GluR1_CKCam > spec_GluR1_S831 + spec_CKCaMCa4, 0.0004)
reaction102 = rxd.Reaction(spec_GluR1 + spec_CKpCaMCa4 != spec_GluR1_CKpCam, 0.0278, 0.002)
reaction103 = rxd.Reaction(spec_GluR1_CKpCam > spec_GluR1_S831 + spec_CKpCaMCa4, 0.0005)
reaction104 = rxd.Reaction(spec_GluR1 + spec_CKp != spec_GluR1_CKp, 0.02224, 0.0016)
reaction105 = rxd.Reaction(spec_GluR1_CKp > spec_GluR1_S831 + spec_CKp, 0.0004)
reaction119 = rxd.Reaction(spec_GluR1_S831 + spec_PP1 != spec_GluR1_S831_PP1, 0.875, 0.0014)
reaction120 = rxd.Reaction(spec_GluR1_S831_PP1 > spec_GluR1 + spec_PP1, 0.00035)

reaction_Ca_flux = rxd.Rate(spec_Ca, Ca_flux_rate)

#Initialize vectors where to store the data
vec_t = h.Vector()
vec_Ca = h.Vector()
vec_CaOut = h.Vector()
vec_CaOutLeak = h.Vector()
vec_Leak = h.Vector()

vec_pmca = h.Vector()
vec_ncx = h.Vector()
vec_pmcaCa = h.Vector()
vec_ncxCa = h.Vector()

vec_CaM = h.Vector()
vec_CaMCa2 = h.Vector()
vec_CaMCa4 = h.Vector()
vec_CK = h.Vector()
vec_CKCaMCa4 = h.Vector()
vec_CKpCaMCa4 = h.Vector()
vec_CKp = h.Vector()
vec_Complex = h.Vector()
vec_pComplex = h.Vector()
vec_CKpPP1 = h.Vector()
vec_CKpCaMCa4PP1 = h.Vector()
vec_PP1 = h.Vector()

vec_GluR1 = h.Vector()
vec_GluR1_S831 = h.Vector()
vec_GluR1_CKCam = h.Vector()
vec_GluR1_CKpCam = h.Vector()
vec_GluR1_CKp = h.Vector()
vec_GluR1_S831_PP1 = h.Vector()

#Record the time (vec_t) and concentrations (vec_<others>) of each species
vec_t.record(h._ref_t)
vec_Ca.record(spec_Ca.nodes(dend)(0.5)[0]._ref_concentration)
vec_CaOut.record(spec_CaOut.nodes(dend)(0.5)[0]._ref_concentration)
vec_CaOutLeak.record(spec_CaOutLeak.nodes(dend)(0.5)[0]._ref_concentration)
vec_Leak.record(spec_Leak.nodes(dend)(0.5)[0]._ref_concentration)

vec_pmca.record(spec_pmca.nodes(dend)(0.5)[0]._ref_concentration)
vec_ncx.record(spec_ncx.nodes(dend)(0.5)[0]._ref_concentration)
vec_pmcaCa.record(spec_pmcaCa.nodes(dend)(0.5)[0]._ref_concentration)
vec_ncxCa.record(spec_ncxCa.nodes(dend)(0.5)[0]._ref_concentration)

vec_CaM.record(spec_CaM.nodes(dend)(0.5)[0]._ref_concentration)
vec_CaMCa2.record(spec_CaMCa2.nodes(dend)(0.5)[0]._ref_concentration)
vec_CaMCa4.record(spec_CaMCa4.nodes(dend)(0.5)[0]._ref_concentration)
vec_CK.record(spec_CK.nodes(dend)(0.5)[0]._ref_concentration)
vec_CKCaMCa4.record(spec_CKCaMCa4.nodes(dend)(0.5)[0]._ref_concentration)
vec_CKpCaMCa4.record(spec_CKpCaMCa4.nodes(dend)(0.5)[0]._ref_concentration)
vec_CKp.record(spec_CKp.nodes(dend)(0.5)[0]._ref_concentration)
vec_Complex.record(spec_Complex.nodes(dend)(0.5)[0]._ref_concentration)
vec_pComplex.record(spec_pComplex.nodes(dend)(0.5)[0]._ref_concentration)
vec_CKpPP1.record(spec_CKpPP1.nodes(dend)(0.5)[0]._ref_concentration)
vec_CKpCaMCa4PP1.record(spec_CKpCaMCa4PP1.nodes(dend)(0.5)[0]._ref_concentration)
vec_PP1.record(spec_PP1.nodes(dend)(0.5)[0]._ref_concentration)
vec_GluR1_S831_PP1.record(spec_GluR1_S831_PP1.nodes(dend)(0.5)[0]._ref_concentration)

vec_GluR1.record(spec_GluR1.nodes(dend)(0.5)[0]._ref_concentration)
vec_GluR1_S831.record(spec_GluR1_S831.nodes(dend)(0.5)[0]._ref_concentration)
vec_GluR1_CKCam.record(spec_GluR1_CKCam.nodes(dend)(0.5)[0]._ref_concentration)
vec_GluR1_CKpCam.record(spec_GluR1_CKpCam.nodes(dend)(0.5)[0]._ref_concentration)
vec_GluR1_CKp.record(spec_GluR1_CKp.nodes(dend)(0.5)[0]._ref_concentration)

#Use adaptive time step. Note that the tolerance generally has to be much smaller than in multicompartmental neuron models
cvode = h.CVode()
cvode.active(1)
hmax = cvode.maxstep(1000)
hmin = cvode.minstep(1e-10)
cvode.atol(tolerance)

#Define a function for setting the Ca2+ influx parameter according to user input
def set_param(param, val):
    param.nodes.value = val
    h.cvode.re_init()


In [4]:
#This is the main simulation function of the RxD system. 
def runneuron(isadaptive=True, Ca_input_flux=2000, Ca_input_N=100):
    my_start_time = time.time()
    h.finitialize()                                           #Initialize the dynamic variables
    cvode.active(isadaptive)

    T = 1000./Ca_input_freq
    tnow = 0
    for istim in range(0,Ca_input_N):
        tnew = Ca_input_onset + istim*T
        h.cvode.event(tnew, lambda: set_param(Ca_flux_rate, Ca_input_flux/6.022e23/my_volume*1e3))
        h.cvode.event(tnew+Ca_input_dur, lambda: set_param(Ca_flux_rate, 0))

    print('Starting simulation...')
    h.continuerun(Duration)                                         #Run the simulation
    print('Simulation run in '+str(time.time() - my_start_time)+' seconds.')

    f,ax = plt.subplots(3,2)
    ax[0,0].plot(array(vec_t)/1000,array(vec_Ca),label='Ca2+')
    ax[0,1].plot(array(vec_t)/1000,array(vec_Ca),label='Ca2+')
    ax[1,0].plot(array(vec_t)/1000,array(vec_CaMCa4),label='CaMCa4')
    ax[1,1].plot(array(vec_t)/1000,array(vec_CaMCa4),label='CaMCa4')
    ax[2,0].plot(array(vec_t)/1000,array(vec_GluR1)+array(vec_GluR1_CKCam)+array(vec_GluR1_CKpCam)+array(vec_GluR1_CKp),label='non-phos.')
    ax[2,1].plot(array(vec_t)/1000,array(vec_GluR1)+array(vec_GluR1_CKCam)+array(vec_GluR1_CKpCam)+array(vec_GluR1_CKp),label='non-phos.')
    ax[2,0].plot(array(vec_t)/1000,array(vec_GluR1_S831)+array(vec_GluR1_S831_PP1),label='S831')
    ax[2,1].plot(array(vec_t)/1000,array(vec_GluR1_S831)+array(vec_GluR1_S831_PP1),label='S831')
    ax[2,0].set_xlabel('$t$ (s)')
    ax[2,1].set_xlabel('$t$ (s)')
    ax[0,0].set_ylabel('$[\mathrm{Ca}^{2+}]$\n(mmol/l)')
    ax[1,0].set_ylabel('$[\mathrm{CaMCa4}]$\n(mmol/l)')
    ax[2,0].set_ylabel('$[\mathrm{AMPAR}]$\n(mmol/l)')
    ax[2,1].legend()
    for i in range(0,3):
      ax[i,0].set_xlim([2950, 4000])
      ax[i,1].set_xlim([2999.5, 3002.5])
      
    plt.show()



In [5]:
      slider = widgets.interact(runneuron, isadaptive=fixed(True),
                                           Ca_input_flux=widgets.FloatSlider(min=0.0,max=3000.0,step=200,value=2000.0,description='Ca2+ flux amplitude',layout=Layout(width='50%'), continuous_update = False),
                                           Ca_input_N=widgets.IntSlider(min=0,max=200,step=2,value=100,description='# stimuli',layout=Layout(width='50%'), continuous_update = False))
                                


interactive(children=(FloatSlider(value=2000.0, continuous_update=False, description='Ca2+ flux amplitude', la…

N.B. This simulation is slow (appr. 20s run-time) as a small tolerance is needed and the convergence to a steady initial state is slow.<br>
Question 1: Vary the magnitude and number of inputs to see the effects on AMPAR phosphorylation.<br>
<b>Question 2</b>: Change the code in function runneuron() to introduce another train of Ca2+ inputs 60 seconds after the first one. Is the effect on AMPAR phosphorylation cumulative?